# Integration
* Exposes a function that trains, optimizes, and saves a neural network model

In [11]:
function train_alpha(hyp::Hyperparams, outdir::String; tune_hyperparams = true)
    set_logging_outdir(outdir)

    if tune_hyperparams
        @info "Optimizing hyperparameters..."
        subsampling_factor = get_subsampling_factor(hyp.model)
        hyp_subset = @set hyp.num_users = Int(round(num_users() * subsampling_factor))
        λ = optimize_hyperparams(hyp_subset; max_evals = 100)
        hyp = create_hyperparams(hyp, λ)
    end

    @info "Training model..."
    m, epochs, validation_loss, hyp = optimize_learning_rate(hyp, 10)
    @info "Trained model loss: $validation_loss"

    if hyp.implicit
        @info "Training calibration layer..."
        m = train_calibration_layers(m, "validation", hyp; rng = Random.Xoshiro(hyp.seed))
    end

    @info "Writing alpha..."
    write_params(
        Dict(
            "m" => m,
            "epochs" => epochs,
            "hyp" => hyp,
            "validation_loss" => validation_loss,
        ),
        outdir,
    )
    write_alpha(hyp, m, outdir)
end;